Skład grupy:
    Paweł Awramiuk
    Anna Chojnowska
    Piotr Grusza
    Szymon Leszczyński

In [ ]:
## Polecenia do instalacji pakietów
# conda install -c conda-forge tensorflow
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Konfiguracje globalne
# Formatowanie liczb w DataFrame
pd.set_option('display.precision', 2)
pd.set_option('display.float_format', '{:.2f}'.format)

2023-11-12 21:50:33.774534: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
data_AUG = pd.read_csv("apartments_pl_2023_08.csv")
data_SEP = pd.read_csv("apartments_pl_2023_09.csv")
data_OCT = pd.read_csv("apartments_pl_2023_10.csv")

In [ ]:
data_AUG.drop(columns=["id"], inplace=True)
data_AUG.drop(columns=["schoolDistance","clinicDistance","postOfficeDistance","kindergartenDistance","restaurantDistance","collegeDistance","pharmacyDistance"], inplace=True)
data_AUG.drop(columns=["buildingMaterial","condition", "type"], inplace=True)

data_SEP.drop(columns=["id"], inplace=True)
data_SEP.drop(columns=["schoolDistance","clinicDistance","postOfficeDistance","kindergartenDistance","restaurantDistance","collegeDistance","pharmacyDistance"], inplace=True)
data_SEP.drop(columns=["buildingMaterial","condition", "type"], inplace=True)

data_OCT.drop(columns=["id"], inplace=True)
data_OCT.drop(columns=["schoolDistance","clinicDistance","postOfficeDistance","kindergartenDistance","restaurantDistance","collegeDistance","pharmacyDistance"], inplace=True)
data_OCT.drop(columns=["buildingMaterial","condition", "type"], inplace=True)


data_SEP.drop_duplicates(inplace = True)
data_AUG.drop_duplicates(inplace = True)
data_OCT.drop_duplicates(inplace = True)

Na podstawie wcześniejszej analizy usuwamy kolumny: "schoolDistance","clinicDistance","postOfficeDistance","kindergartenDistance","restaurantDistance","collegeDistance","pharmacyDistance", gdyż są one silnie ze sobą skorelowane, a w danych występuje dodatkowa zależna od nich kolumna "poiCount". Usuwamy również kolumnę "id" zawierającą unikatowy identyfikator wiersza, oraz ze względu na wysoki procent braków w rekordach kolumny "buildingMaterial", "condition" oraz "type".

In [ ]:
missing_values = pd.concat([
        data_AUG.isna().sum(),
        data_AUG.isna().sum() / len(data_AUG) * 100,
        data_SEP.isna().sum(),
        data_SEP.isna().sum() / len(data_SEP) * 100,
        data_OCT.isna().sum(),
        data_OCT.isna().sum() / len(data_OCT) * 100
    ],
    axis=1
)
missing_values.rename(columns = {0:'AUG', 1:'AUG %',2:'SEP', 3:'SEP %',4:'OCT', 5:'OCT %',6:'ALL', 7:'ALL %'}, inplace = True)
missing_values

In [ ]:
data_AUG.isna().sum(1).hist()

In [ ]:
data_SEP.isna().sum(1).hist()

In [ ]:
data_OCT.isna().sum(1).hist()

In [ ]:
data_AUG = data_AUG.loc[(data_AUG.isna().sum(axis=1)<3)]
data_SEP = data_SEP.loc[(data_SEP.isna().sum(axis=1)<3)]
data_OCT = data_OCT.loc[(data_OCT.isna().sum(axis=1)<3)]

Ze względu na niski odsetek rekordów posiadajacych więcej, niż dwa braki, oferty takie zostały usunięte, celem łatwiejszego uzupełnienia braków w ofertach.

In [ ]:
def show_missing_values(data):
    missing_values = pd.concat([
        data.isna().sum(),
        data.isna().sum() / len(data) * 100
    ],axis=1)
    missing_values.rename(columns = {0:'MISSING', 1:'MISSING %'}, inplace = True)
    print(missing_values)

In [ ]:
missing_values = pd.concat([
        data_AUG.isna().sum(),
        data_AUG.isna().sum() / len(data_AUG) * 100,
        data_SEP.isna().sum(),
        data_SEP.isna().sum() / len(data_SEP) * 100,
        data_OCT.isna().sum(),
        data_OCT.isna().sum() / len(data_OCT) * 100
    ],
    axis=1
)
missing_values.rename(columns = {0:'AUG', 1:'AUG %',2:'SEP', 3:'SEP %',4:'OCT', 5:'OCT %',6:'ALL', 7:'ALL %'}, inplace = True)
missing_values

In [ ]:
import missingno as msno

In [ ]:
msno.heatmap(data_AUG)

In [ ]:
msno.heatmap(data_SEP)

In [ ]:
msno.heatmap(data_OCT)

Powyższe wykresy wskazują na to, że nie występuje korelacja pomiędzy brakującymi danymi w poszczególnych miesiąca, co może oznaczać, że mamy do czynienia z brakami "Missing at Random".

## Przypisanie miesiąca do poszczególnych ofert na podstawie pliku, z którego pochodzą dane

In [ ]:
# dodanie miesiąca, w którym dana oferta sprzedaży była dostępna (wszystkie dane pochodzą z roku 2023)
data_AUG["offerMonth"] = 8
data_SEP["offerMonth"] = 9
data_OCT["offerMonth"] = 10

Do każdej oferty dodajemy informację o miesiącu, z którego pochodzi, co po złączeniu plików da nam informację o trendach zmian cen w danym przedziale czasu.

# Obsługa brakujących danych

W przypadku brakujących danych, które zostaną uzupełnione wartościami średnimi lub medianą, operacje zostaną wykonane oddzielnie w obrębie poszczególnych miesięcy, aby uzupełnione oferty, były bardziej zbliżone do innych ofert wystawionych w podobnym czasie.

Uzupełnienie danych w kolumnach, w których wartości zostaną uzupełnione na podstawie innych kolumn/wiedzy dziedzinowej zostanie wykonane po wcześniejszym połączeniu danych w jeden DataFrame.

## buildYear, floorCount, floor

Braki w tych trzech zmiennych są uzupełniane medianą.

In [ ]:
data_AUG.fillna(data_AUG.median(numeric_only=True), inplace=True)
data_SEP.fillna(data_SEP.median(numeric_only=True), inplace=True)
data_OCT.fillna(data_OCT.median(numeric_only=True), inplace=True)

## Złączenie danych

In [ ]:
data_ALL = pd.concat([data_AUG, data_SEP, data_OCT], ignore_index=True)

In [ ]:
data_ALL.tail()

In [ ]:
data_ALL.count()

## hasElevator

https://isap.sejm.gov.pl/isap.nsf/download.xsp/WDU20190001065/O/D20191065.pdf
**Dział III Rozdział 1 Paragraf 54**

In [ ]:
print("hasElevator NA ALL:" + str(data_ALL["hasElevator"].isna().sum()))

Budynki mieszkalne mające więcej niż 4 piętra wymagają windy

In [ ]:
filt = (data_ALL["floorCount"] > 4)
new_val = data_ALL.loc[filt, ["hasElevator"]].fillna("yes", axis=1)
data_ALL.loc[filt, ["hasElevator"]] = new_val

W budynkach mających 4 lub mniej pięter parametr określający obecność windy ustawiany jest na wartość: "nie"

In [ ]:
filt = ((data_ALL["floorCount"] <= 4) & (data_ALL["floorCount"].notnull()))
new_val = data_ALL.loc[filt, ["hasElevator"]].fillna("no", axis=1)
data_ALL.loc[filt, ["hasElevator"]] = new_val

In [ ]:
print("hasElevator NA ALL:" + str(data_ALL["hasElevator"].isna().sum()))

## Ownership

### Wartość odstająca

In [ ]:
data_ALL[data_ALL["ownership"] == "udział"]

In [ ]:
data_ALL.drop(data_ALL[data_ALL["ownership"] == "udział"].index, inplace=True)

Występuje tylko 1 rekord z wartością ownership="udział" z tego powodu zostaje usunięty.

### One-hot encoding

In [ ]:
data_ALL = pd.get_dummies(data_ALL, columns=["ownership"], prefix="ownership")

Ownership jest zmienną tekstową, dlatego zostaje zakodowana metodą one-hot.

## City - one-hot encoding

"city" jest zmienną teskstową kategoryczną o liczności 15. Z tego powodu zostanie zakodowana metodą one-hot.

In [ ]:
data_ALL = pd.get_dummies(data_ALL, columns=["city"], prefix="city")
data_ALL

## Konwersja zmiennych 2-wartościowych na typ boolean

In [ ]:
data_ALL.info()

In [ ]:
columns_to_convert = ["hasParkingSpace", "hasBalcony", "hasElevator", "hasSecurity", "hasStorageRoom"]

In [ ]:
for i in columns_to_convert:
    data_ALL[i] = data_ALL[i].map({'yes': 1, 'no': 0})

In [ ]:
data_ALL[columns_to_convert]

In [ ]:
data_ALL.info()

## Analiza odchyłek

In [ ]:
cols_out = ["squareMeters", "rooms", "floor", "floorCount", "buildYear", "latitude", "longitude","centreDistance","poiCount"]

In [ ]:
data_ALL[cols_out].hist()

In [ ]:
data_ALL[cols_out].quantile([0, 0.25, 0.5, 0.75, 1.])

Odchyłki są wykrywane metodą IQR - za odchyłkę (outlayer) uznajemy rekord, którego wartość leży o półtora różnicy między 1. i 3. kwartylem powyżej 3. lub poniżej 1. kwartyla

In [ ]:
iqr = {}
for col in cols_out:
    iqr[col] = (data_ALL[col].quantile(0.75) - data_ALL[col].quantile(0.25)) * 1.5
    
iqr

In [ ]:
data_ALL_clear = data_ALL
for col in cols_out:
    q1 = data_ALL[col].quantile(0.25)
    q3 = data_ALL[col].quantile(0.75)
    data_ALL_clear = data_ALL_clear.loc[(data_ALL_clear[col] >= q1 - iqr[col]) & (data_ALL_clear[col] <= q3 + iqr[col])]
    
data_ALL = data_ALL_clear
data_ALL

In [ ]:
data_ALL[cols_out].hist()

In [ ]:
data_ALL[cols_out].quantile([0, 0.25, 0.5, 0.75, 1.])

## Normalizacja kolumn numerycznych

In [ ]:
cols_to_norm = ["squareMeters", "rooms", "floor", "floorCount", "buildYear", "latitude", "longitude","centreDistance","poiCount"]

In [ ]:
for col in cols_to_norm:
    mean = data_ALL[col].mean()
    std = data_ALL[col].std()
    data_ALL[col] = (data_ALL[col]-mean)/std

In [ ]:
data_ALL[cols_to_norm]